### Разработка тестового приложения
#### Построение sql-запросов и графиков

Основное назначение созданной базы данных - возможность формирования сводной аналитики в разрезе различных групп студентов, а именно отчисленных из образовательного учреждения и соответственно не отчисленных. Предполагается, что на основе базы данных студентов возможно более эффективное и быстрое получение сведений о студентах как относительно их персональных характеристик, так и окружающего их контекста, создавать информативные визуализации на их основе. Помимо этого, созданная база может служить основой и для реализации анализа данных с уклоном в методы машинного обучения, поскольку занесенные в базу характеристики возможно переформировать в переменные для использования в соответствующих моделях, например, моделях линейной или логистической регрессий (логистической регрессии для прогнозирования условий, при которых студент будет отчислен).

В рамках данного отчете фокус был сделан именно на визуализации данных, выгруженных с помощью sql-запросов из базы. В частности, были созданы такие визуализации как:

1) Круговые диаграммы в разрезе групп отчисленных/не отчисленных студентов относительно качества взаимоотношений с родителями;

2) Круговые диаграммы в разрезе групп отчисленных/не отчисленных студентов относительно качества состояния здоровья;

3) Столбиковая диаграмма с выводом топа школ по % отчисленных студентов относительно общего количества учащихся в учебном заведении;

4) Боксплоты с репрезентацией распределения количественных характеристик - количества пропущенных занятий и количества несданных экзаменов в разрезе отчисленных/не отчисленных групп студентов.

Визуализации выполнялись с помощью библиотеки plotly. Всего было создано 4 графика с опорой на данные полученные из трех запросов к базе.

Для выполнения каждой из визуализаций был осуществлен следующий пайплайн работы:
1) написание sql-запроса к базе данных, обработка исключений и ошибок при выгрузке данных из базы и их оформлении в датафрейм;

2) работа с выгруженным датафреймом относительно добавления дополнительных меток, агрегации, сортировки, мерджа данных при необходимости;

3) визуализация полученных данных с помощью библиотеки Plotly.

Выполненные визуализации позволили более эффективно изучить не только структуру, но и содержательные особенности данных, выявить наличие выраженных эмпирических взаимосвязей между исследуемыми характеристиками и группами студентов, относительно которых они рассматриваются.

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors
import pandas as pd

import sqlite3
import sqlalchemy

#### Пример редактирования данных в БД

In [4]:
### РЕДАКТИРОВАНИЕ ДАННЫХ В БД
### на примере смены названия одного из предметов (дополнения названия предмета Economics - Economics & Social Sciences)
### редактирование в т.ч. осуществляется с опорой на обработку исключений

students_db = sqlite3.connect('students.db')
cursor = students_db.cursor()

try:
    cursor.execute(f"UPDATE Courses SET name = ? WHERE name = ?", 
                   ('Economics & Social Sciences', 'Economics'))
    students_db.commit()
    print("Values in table were successfully updated!")
except Exception as e:
    print(f"An error occurred {e}")

Values in table were successfully updated!


#### Контроль работы ограничений целостности

In [10]:
# проверка на ограничение относительно первичного ключа в таблице со студентами
try:
    cursor.execute(f"INSERT INTO Students (id) VALUES (?)", 
                   (2,))
    students_db.commit()
    print("Values were successfully inserted into table!")
except Exception as e:
    print(f"An error occurred: {e}")


# проверка на ограничение относительно запрета на ввод 0 в кол-ве учеников в школе
try:
    cursor.execute(f"INSERT INTO Schools (number_of_students) VALUES (?)", 
                   (0,))
    students_db.commit()
    print("Values were successfully inserted into table!")
except Exception as e:
    print(f"An error occurred: {e}")
    

# проверка на ограничение относительно ввода допустимых значений в поле из определенного "списка"
try:
    cursor.execute(f"INSERT INTO HealthStatuses (state_of_health) VALUES (?)", 
                   ('not very good',))
    students_db.commit()
    print("Values were successfully inserted into table!")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: UNIQUE constraint failed: Students.id
An error occurred: CHECK constraint failed: number_of_students > 0
An error occurred: CHECK constraint failed: state_of_health IN ('Poor', 'Average', 'Fair', 'Good', 'Excellent')


#### Выполнение запросов к базе и отрисовка графиков

Запросы к базе выполнялись с опорой на библиотеку sqlalchemy, т.к. запрос исключительно средствами библиотеки sqlite3 не позволил получить названия полей напрямую, в выдаче были только индексы полей, соответственно для повышения эффективности и скорости работы с данными (в частности, для того, чтобы не приходилось вручную присваивать названия полей), была использована библиотека sqlalchemy. Для каждого запроса была предусмотрена обработка исключений.

In [37]:
# далее запросы к базе данных совершаются с опорой на sqlalchemy, т.к. простой запрос 
# через sqlite3 напрямую не позволяет получить названия полей (вместо этого им присваются индексы)

# были выгружены характеристики относительно ментального здоровья студентов, факта их отчисления

connection_to_db = sqlalchemy.create_engine("sqlite:///students.db")
with connection_to_db.connect() as connection:
    try:
        connection.execute(sqlalchemy.text("PRAGMA foreign_keys = ON"))
        query = sqlalchemy.text('''
                                SELECT Students.id,  
                                DropoutsRecords.student_id drop_id,
                                MentalHealthStatuses.romantic_relat, 
                                MentalHealthStatuses.family_relat_score,
                                FamilyRelationship.description,
                                HealthStatuses.state_of_health
                                FROM Students 
                                LEFT JOIN DropoutsRecords
                                ON Students.id = DropoutsRecords.student_id
                                LEFT JOIN MentalHealthStatuses
                                ON Students.id = MentalHealthStatuses.student_id
                                INNER JOIN FamilyRelationship
                                ON MentalHealthStatuses.family_relat_score = FamilyRelationship.id
                                INNER JOIN HealthStatuses
                                ON Students.id = HealthStatuses.student_id''')
        df = pd.read_sql_query(query, connection)
        print('Data successfully loaded from DB')
    except Exception as e:
        print(f'Some errors occured during connection to db and data fetching: {e}')

# присвоение дополнительной метки на основе выгруженных из базы данных
# соответственно, где drop_id содержит NaN, студент не зафикисрован в качестве отчисленного
# для удобства визуализации возможно перемаркировать данные - создав другой столбец

df.loc[(df['drop_id'].isnull() == True), 'drop_marking'] = 'not_dropped_out'
df.loc[(df['drop_id'].isnull() != True), 'drop_marking'] = 'dropped_out'

Data successfully loaded from DB


In [38]:
# агргегация данных и отрисовка круговых диаграмм
students_groups = pd.DataFrame(df.groupby(['drop_marking', 'description'])['id'].count()).reset_index()
students_groups_dropped = students_groups[students_groups['drop_marking'] == 'dropped_out']
students_groups_not_dropped = students_groups[students_groups['drop_marking'] == 'not_dropped_out']

fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type': 'pie'}, {'type': 'pie'}]],
                    subplot_titles=['''Dropped out students''', 'Not dropped out students'])

fig.add_trace(go.Pie(labels=students_groups_dropped['description'], 
                     values=students_groups_dropped['id'], 
                     name='Dropped out students', hole=0.5), 
                     row=1, col=1)

fig.add_trace(go.Pie(labels=students_groups_not_dropped['description'], 
                     values=students_groups_not_dropped['id'], 
                     name='Not dropped out students', hole=0.5), 
                     row=1, col=2)


fig.update_layout(
    title_text="<b>Familiy relationship score among groups of students<b>",
    height=600,
    showlegend=True,
    legend=dict(
        orientation='h',
        y=-0.2,
        xanchor="right",
        x=1
    ),
)

fig.show()

# на основании полученной диаграммы представляется возможным сделать вывод о том, что в группе отчисленных студентов 
# процент учащихся, у которых слабые или крайне слабые взаимоотношения с семьей/родителями, выше (14%), чеи для группы
# студентов, которые не были отчислены из учебного заведения (около 7%)

# так же относительно групп выражено отличие относительно процента студентов, у которых очень близкие взаимоотношения с родителями,
# в частности, для группы студентов, которые отчислены не были, этот процент (около 52%) выше, чем для противоположной группы учащихся (34%)

In [39]:
# тот же самый вид диаграмм, но для другой характеристики - состояния здоровья студентов

students_groups = pd.DataFrame(df.groupby(['drop_marking', 'state_of_health'])['id'].count()).reset_index()
students_groups_dropped = students_groups[students_groups['drop_marking'] == 'dropped_out']
students_groups_not_dropped = students_groups[students_groups['drop_marking'] == 'not_dropped_out']

fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type': 'pie'}, {'type': 'pie'}]],
                    subplot_titles=['''Dropped out students''', 'Not dropped out students'])

fig.add_trace(go.Pie(labels=students_groups_dropped['state_of_health'], 
                     values=students_groups_dropped['id'], 
                     name='Dropped out students', hole=0.5), 
                     row=1, col=1)

fig.add_trace(go.Pie(labels=students_groups_not_dropped['state_of_health'], 
                     values=students_groups_not_dropped['id'], 
                     name='Not dropped out students', hole=0.5), 
                     row=1, col=2)


fig.update_layout(
    title_text="<b>State of health among groups of students<b>",
    height=600,
    showlegend=True,
    legend=dict(
        orientation='h',
        y=-0.2,
        xanchor="right",
        x=1
    ),
)

fig.show()

# относительно анализируемого показателя отсутствуют выраженные отличия между студентами из разных групп
# процент студентов с "крайне низким" состоянием здоровья в группе отчисленных несколько выше (17%), чем для
# противоположной группы (13%); так же среди неотчисленных студентов процент учащихся с "хорошим" состоянием здоровья
# также несколько выше (около 18% против 12%)

In [40]:
# выгружены характеристики относительно студентов (в разрезе факта их отчисления), а также
# относительно учебных заведения

with connection_to_db.connect() as connection:
    try:
        connection.execute(sqlalchemy.text("PRAGMA foreign_keys = ON"))
        query = sqlalchemy.text('''
                                SELECT Students.id,
                                DropoutsRecords.student_id drop_id,
                                Schools.name, Schools.status,
                                SchoolsStatus.description,
                                Schools.number_of_students
                                FROM Students 
                                LEFT JOIN DropoutsRecords
                                ON Students.id = DropoutsRecords.student_id
                                INNER JOIN Schools
                                ON Students.school_id = Schools.id 
                                INNER JOIN SchoolsStatus
                                ON Schools.status = SchoolsStatus.id 
                                ''')
        df = pd.read_sql_query(query, connection)
        print('Data successfully loaded from DB')
    except Exception as e:
        print(f'Some errors occured during connection to db and data fetching: {e}')
        

df.loc[(df['drop_id'].isnull() == True), 'drop_marking'] = 'not_dropped_out'
df.loc[(df['drop_id'].isnull() != True), 'drop_marking'] = 'dropped_out'

Data successfully loaded from DB


In [41]:
#поиск топ 10 школ по количеству отчисленных студентов (в % от общего количества учащихся)
schools_info = pd.DataFrame(df.groupby(['drop_marking', 'name'])['id'].count()).reset_index()
schools_info = schools_info[schools_info['drop_marking'] == 'dropped_out']


schools_info_students_num = df[['name', 'number_of_students']].drop_duplicates()
schools_info_merged = schools_info.merge(schools_info_students_num, on = 'name', how = 'left')
schools_info_merged['%_of_dropped_out'] = round((schools_info_merged['id']/schools_info_merged['number_of_students'])*100,2)
schools_info_merged = schools_info_merged.sort_values(by = '%_of_dropped_out')
schools_info_merged = schools_info_merged[:10]
display(schools_info_merged)

,drop_marking,name,id,number_of_students,%_of_dropped_out
34,dropped_out,New Jasmine Junior School,1,581,0.17
39,dropped_out,New Thomas School,1,598,0.17
37,dropped_out,New Robertport Junior School,1,559,0.18
71,dropped_out,West Thomasshire Middle School,1,566,0.18
38,dropped_out,New Shannonhaven School,1,569,0.18
49,dropped_out,North Monicaburgh Junior School,1,569,0.18
10,dropped_out,Davidshire School,1,561,0.18
16,dropped_out,Elizabethburgh Junior School,1,554,0.18
52,dropped_out,Port Brenda Middle School,1,532,0.19
19,dropped_out,Jonathantown Middle School,1,513,0.19


In [42]:
# отображение полученных данных на графике (столбчатой диаграмме)
fig = go.Figure(data=[go.Bar(y=schools_info_merged['name'], 
                             x=schools_info_merged['%_of_dropped_out'], 
                             orientation='h',
                             marker_color='#C71585')],
                            )

# Update layout
fig.update_layout(title_text="<b>Number of dropped out students per school (in %)<b>",
                   yaxis_title="Schools",
                   xaxis_title="% of dropped out students")


fig.show()

# лидерами по количеству отчисленных студентов (в % от общего числа учащихся в школе) стали такие учебные заведения как:
# Jonathantown Middle School и Port Brenda Middle School

In [43]:
# выгрузка данных об академических характеристиках студентов, в т.ч. с добавление данных об факте их отчисления
with connection_to_db.connect() as connection:
    try:
        connection.execute(sqlalchemy.text("PRAGMA foreign_keys = ON"))
        query = sqlalchemy.text('''
                                SELECT Students.id,
                                DropoutsRecords.student_id drop_id,
                                AcademicPerfomances.absence_num,
                                AcademicPerfomances.failures_num
                                FROM Students 
                                LEFT JOIN DropoutsRecords
                                ON Students.id = DropoutsRecords.student_id
                                INNER JOIN AcademicPerfomances
                                ON Students.id = AcademicPerfomances.student_id 
                                ''')
        df = pd.read_sql_query(query, connection)
        print('Data successfully loaded from DB')
    except Exception as e:
        print(f'Some errors occured during connection to db and data fetching: {e}')
        
df.loc[(df['drop_id'].isnull() == True), 'drop_marking'] = 'not_dropped_out'
df.loc[(df['drop_id'].isnull() != True), 'drop_marking'] = 'dropped_out'

Data successfully loaded from DB


In [45]:
# получение данных о группах и о двух количественных характеристиках - количестве пропусков занятий и количестве несданных экзаменов
groups_of_students = df['drop_marking'].unique()

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=(f'Number of absence ({groups_of_students[0]})', f'Number of absence ({groups_of_students[1]})',
                                    f'Number of failures ({groups_of_students[0]})', f'Number of failures ({groups_of_students[1]})')
                   )

for i, label in enumerate(groups_of_students):
    group_data = df[df['drop_marking'] == label]
    fig.add_trace(go.Box(y=group_data['absence_num'], name=f"{label}", showlegend=False), row=1, col=i + 1)
    fig.add_trace(go.Box(y=group_data['failures_num'], name=f"{label}", showlegend=False), row=2, col=i + 1)


fig.update_layout(title_text="<b>Distribution of number of failures and absence among groups of students<b>",
                  height=700,
                  yaxis_title='Count',
                  )


fig.show()

# было принято решение отобразить именно распределение данных показателей относительно двух групп студентов
# так, для группы неотчисленных студентов граница 75 перцентиля количества пропусков 
# расположена ниже (5,25), чем для противоположной группы студентов (8), что говорит о том, что более 25% студентов 
# из числа неотчисленных имеют меньшее количество пропусков, чем другая группа; медианные значения показателя 
# также незначительно отличаются (2 против 2,5);
# интересным аспектом также является и то, что максимальное количество пропусков в выборке в целом принадлежит именно группе 
# неотчисленных студентов (32) против 26 пропусков в противоположной группе; в целом для группы неотчисленных студентов характерно
# большее кол-во выбросов за счет того, что для данной подвыборки студентов более экстремальные значения кол-ва пропусков менее вероятны,
# чем для другой подвыборки

 
# аналогичная ситуация имеет место для другого показателя - количества проваленных экзаменов - для подвыборки неотчисленных студентов
# данное событие является менее вероятным, поэтому весь бокслот представляет собой сплошную линию в районе 0 и несколько выбросов над ней;
# так, в группе неотчисленных есть три студента, которые не сдавали экзамен 1,2 или 3 раза подряд, но все равно не были отчислены;
# иное положение характерно для противоположной группы, в которой более 25% студентов имеют хотя бы 1 несданный экзамен, однако ситуация с 
# тремя несданными экзаменами так же является редкой для данной групп (такой результат только у одного студента из выборки)

Таким образом, благодаря тому, что из базы данных возможно делать комплексные выгрузки - с включением параметров из разных таблиц, выполнение визуализации становится более быстрой задачей. В частности, подобные запросы и визуализации могут выполняться в качестве эксплораторного анализа данных (EDA), чтобы более предметно ознакомиться с выборкой, в т.ч. в разрезе различных групп. Без базы данных выполнение аналогичных задач заняло гораздо большее количество времени, в частности, за счет осуществления шагов мерджа разных таблиц, переименования полей для осуществления мерджа (т.к. таблицы не имели бы четко определенных ключей), валидации данных (проверка корректности введенных данных, их редактирование прямо в коде при необходимости). Оформление сведений о студентах в единую базу данных нивелирует эти сложности и позволяет более гибко и эффективно работать с данными, поскольку в ней уже установлены соответствующие отношения между таблицами через ключи и в ряде случаев при необходимости реализована валидация данных, вносимых в базу.